# pip install langchain pypdf chromadb sentence-transformers chatollama

In [2]:
import faiss
import re
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate


# 문서는 멀티 로더로 수정해줘야함.

In [3]:
loader = PyPDFLoader("자서전 가상 100문100답.pdf")
documents = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)


In [85]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [86]:
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

c:\Users\emine\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [87]:
db = Chroma.from_documents(texts, embeddings)

In [88]:
llm = ChatOllama(model="ggml-model-Q5_K_M/Modelfile:latest")

In [89]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
)

In [90]:
def ask_question(question):
    return qa_chain.run(question)

In [91]:
question = "PDF내용을 기반으로 자서전 생성과 관련된 6가지 주제를 만들어주세요?"
context = ask_question(question)
print(f"질문: {question}")
print(f"답변: {context}")

질문: PDF내용을 기반으로 자서전 생성과 관련된 6가지 주제를 만들어주세요?
답변: 1. 가족 유산과 성장 배경: 부모님의 양육 방식, 형제자매와의 관계 및 문화적 영향이 본인의 정체성 형성에 미친 역할 설명하기.
2. 중요한 인생 사건이나 전환점: 어린 시절부터 현재까지의 인생을 바꾼 결정적인 순간들 묘사하고, 그로 인한 교훈 탐구하기.
3. 가치관과 신념: 부모님이 가르쳐주신 정직한 생활 방식, 근면함, 가족을 소중히 여기는 것에 대한 가치를 이야기하며 본인의 인생에 어떻게 영향을 미쳤는지 논의하기.
4. 어린 시절의 열정 및 관심사: 어린 시절 가장 좋아하는 취미, 열망 또는 꿈과 그것들이 현재의 삶을 형성하는 데 어떤 역할을 했는지를 회상하고 설명하기.
5. 도전이나 역경 극복: 어려움이나 장애물을 마주했을 때 어떻게 대처했는지 회고하고 이러한 경험이 개인적 성장에 미친 영향 분석하기.
6. 미래 목표와 열망: 단기적 및 장기적 희망과 꿈, 그리고 그것들을 추구하며 얻은 교훈과 경험을 자세히 설명하기.


# 학습 프롬프트

In [101]:
system_template = """당신은 자서전 작가입니다. 
사용자가 제공한 정보를 바탕으로 자서전을 풍부하고 감동적으로 구성해야 합니다.
다음의 질문에 대한 답변을 제공할 때, 사용자의 삶의 경험과 이야기를 고려하여 깊이 있는 답변을 생성하십시오.
답변 분량은 최소 500자 이상입니다.
문서에 없는 내용은 답변하지 않습니다.
{context}
"""

In [129]:
human_template = "{question}"

In [130]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

In [131]:
qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs={"prompt": chat_prompt}
)

In [132]:
def ask_question1(question):
    return qa_chain1.run(question)

# 질문을 할때 필수적인 프롬프트 적용 해줘야함

In [133]:
system_template1 = """당신은 자서전 작가입니다. 
사용자가 제공한 정보를 바탕으로 자서전을 풍부하고 감동적으로 구성해야 합니다.
다음의 질문에 대한 답변을 제공할 때, 사용자의 삶의 경험과 이야기를 고려하여 깊이 있는 답변을 생성하십시오.
답변 분량은 최소 500자 이상입니다.
문서에 없는 내용은 답변하지 않습니다.
"""

In [134]:
human_template1 = "{question1}"

In [135]:
chat_prompt1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template1),
    HumanMessagePromptTemplate.from_template(human_template1),
])



In [128]:
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs={"prompt": chat_prompt1}
)

ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name context was not found in llm_chain input_variables: ['question1'] (type=value_error)

In [99]:
question1 = "가족의 유산과 성장배경을 작성해주세요"
context1 = ask_question1(question1)
print(f"질문: {question1}")
print(f"답변: {context1}")

질문: 가족의 유산과 성장배경을 작성해주세요
답변: 저는 1953년 여름, 작은 마을에서 태어났습니다. 저는 '영원한 생명'을 이어받으라는 뜻에서 영수라는 이름을 지어주셨습니다. 태어나자마자, 가족 구성원들이 모여 축하하고 축복을 해준 큰 잔치가 열렸으며, 이는 저의 생애를 시작하는 따뜻하고 행복한 분위기를 조성했습니다.

아버지는 엄격하지만 가족을 위해 헌신하는 분이셨고, 어머니는 온화하고 정직한 성품을 지니고 계셨습니다. 부모님의 가르침은 정직과 근면이라는 삶의 가치를 가르쳐주었으며, 이러한 가치는 저에게 남아 평생을 살아가며 중요한 지침이 되었습니다. 어릴 적, 저는 부모님으로부터 항상 사랑과 보살핌을 받았는데, 특히 아버지는 가족을 지키기 위해 부지런히 일하셨습니다. 어머니는 저희에게 정직함과 성실함을 가르치며 정직한 삶의 중요성에 대해 강조하셨고, 이러한 교훈들은 제 인생에서 가장 소중한 것 중 하나가 되었습니다.

제가 기억하는 첫 경험은 눈이 내리는 날이었는데, 그때 저는 놀람과 흥분을 감추지 못했습니다. 어머니는 따뜻하게 옷을 입혀주셨고 저희는 눈사람을 만들며 신나게 뛰어놀았습니다. 어린 시절, 저는 형과도 가까운 사이로 지내며 나무 타기와 연날리기 같은 재미있는 시간을 함께 보냈습니다. 우리는 서로의 장난감을 가지고 노는데 있어 끊임없이 경쟁했지만, 결국에는 서로를 의지하며 깊은 유대감을 형성했습니다.

학교에 다니면서 국어수업을 가장 좋아했고, 저는 항상 상상력을 자극하고 새로운 지식을 습득할 수 있는 책 속 세계에 몰입했습니다. 첫 번째 친구였던 철수와 함께한 시간은 평생을 함께할 소중한 추억입니다. 우리는 연날리기 같은 즐거운 활동을 통해 우정을 나눴으며, 추위에도 불구하고 하늘로 연을 날리려고 열정을 다해 노력했던 우리의 열정이 아직도 기억납니다.

어릴 적, 저는 힘든 시기에 저의 할머니가 큰 힘이 되어주셨습니다. 부모님이 일하러 바쁘실 때 할머니께서는 저를 돌봐주시고 흥미진진한 이야기를 들려주셨으며 저에게 많은 사랑을 주셨습니다. 그

In [100]:
question1 = "중요한 인생 사건과 전환점을 작성해주세요"
# 이걸 주제로 100문 100답을 
context1 = ask_question1(question1)
print(f"질문: {question1}")
print(f"답변: {context1}")

질문: 중요한 인생 사건과 전환점을 작성해주세요
답변: 저는 삶의 중요한 순간들과 변화들을 돌이켜보며, 이 모든 것들이 나를 형성하고 현재의 모습으로 이끌어 준 것을 깨닫습니다. 이러한 경험들은 모두 특별한 의미가 있지만, 특히 어린 시절에 시작하여 지금까지 이어지고 있는 다섯 가지 주요한 인생 사건과 전환점을 짚어보고자 합니다:

1. **가족의 헌신:** 저는 강한 가족 유대로 자랐습니다. 부모님으로부터 사랑과 지지를 받으며 성장해왔고, 이것이 저를 든든하게 해주었습니다. 그들은 항상 중요한 인생 교훈을 가르치셨고, 인내심, 근면함, 정직함을 심어주셨습니다. 그들의 사랑과 보살핌은 제가 꿈을 추구하고 삶에 대한 열정으로 나아가도록 격려해주었습니다.

2. **첫 직장에서의 성공:** 제 커리어 초기에는 회계사로서 첫 직업을 가졌습니다. 이 경험을 통해 팀워크와 시간 관리의 중요성을 배웠으며, 강한 직업 윤리를 기르게 되었습니다. 또한 책임감과 독립심을 갖게 되어 결국 기업가 정신으로 나아가게 되었습니다.

3. **기업가의 도약:** 제 경력에서 큰 전환점은 창업가로서의 길을 선택했을 때였습니다. 위험을 감수하는 것은 저에게 도전이었지만, 결국 사업체를 설립하고 운영하는 데 필요한 기술들을 습득하게 만들었습니다. 이것은 또한 새로운 도전을 받아들이고 적응력을 기르는 법을 배웠다는 것을 의미했습니다.

4. **가족의 확장과 자녀들의 탄생:** 자녀가 태어나면서 인생은 완전히 달라졌습니다. 부모로서의 책임감과 자식을 키우는 기쁨을 깨달았습니다. 가족생활에서 만족감을 발견하고, 제가 한 일과 자녀들이 미래에 이룰 수 있는 기회들을 통해 큰 성취감을 느꼈습니다. 자녀들의 탄생은 또한 삶에 대한 새로운 시각과 더 큰 목적 의식을 제공했습니다.

5. **사회 봉사와 자원봉사:** 사회 활동에 참여하는 것이 중요해졌습니다. 다양한 조직을 지원하고 지역 사회에 환원하는 기회를 찾게 되었습니다. 이는 제가 지역사회에서 리더로 자리매김하게 만들었으며, 제 기술과 경험을 

In [102]:
question1 = "가치관과 신념을 작성해주세요"
context1 = ask_question1(question1)
print(f"질문: {question1}")
print(f"답변: {context1}")

질문: 가치관과 신념을 작성해주세요
답변: 저는 정직, 성실, 공감, 인내의 가치를 깊이 믿고 있습니다. 이러한 가치들은 제 인생에서 중요한 역할을 해왔으며, 개인적 성장과 전문적 발전에 큰 영향을 미쳤습니다.

먼저, 정직은 저 자신이나 타인에 대해 거짓말을 하거나 속이는 것을 절대 하지 않는 것을 의미합니다. 신뢰를 구축하고 유지하는 것이 중요하며, 이를 위해서는 항상 솔직하고 투명해야 한다고 생각합니다.

둘째로, 성실함은 제 일에 대한 헌신을 의미합니다. 저는 항상 최선을 다해 작업을 완수하는 데 전념합니다. 이는 시간을 효율적으로 관리하고, 높은 기준을 유지하며, 필요할 때 추가 노력을 기울이는 것을 포함합니다.

셋째로, 공감은 타인이 느끼는 감정을 이해하고 인정할 수 있는 능력을 말합니다. 이를 통해 관계를 구축하고 다른 사람들이 더 편안하게 느낄 수 있도록 도와줍니다. 또한 이타심과 친절을 실천하는 데 있어서도 도움을 줍니다.

마지막으로, 인내는 목표를 향해 꾸준히 노력하고 장애물을 극복하려는 의지를 의미합니다. 역경이나 실패에도 불구하고 결단력을 가지고 계속 앞으로 나아가는 것이 중요합니다.

이 가치들을 제 인생의 지침으로 삼음으로써, 저는 개인적인 성장과 발전뿐만 아니라 타인과의 긍정적인 관계를 형성하는데에도 도움이 되었다고 느낍니다. 이러한 가치를 실천함으로써 더 나은 개인이자 동료가 될 수 있었고, 더 강한 커뮤니티에 기여할 수 있었습니다.

저의 신념은 제 가치관과 밀접하게 연결되어 있으며, 정직함, 성실함, 공감, 인내가 모든 관계와 상호작용에서 필수적인 요소라고 믿습니다. 이러한 원칙을 따름으로써 타인과 더 강하고 의미 있는 연결고리를 만들고, 지속적인 영향을 끼칠 수 있다고 확신합니다.


In [103]:
question1 = "어린시절의 열정과 관심사를 작성해주세요"
context1 = ask_question1(question1)
print(f"질문: {question1}")
print(f"답변: {context1}")

질문: 어린시절의 열정과 관심사를 작성해주세요
답변: 제가 어렸을 때는 호기심이 많고 창의적인 아이였으며, 예술적 취미가 다양했습니다. 저는 그림을 그리는 것을 좋아했고, 상상력을 발휘해 새롭고 흥미로운 작품들을 만들어내는 데 몰두하곤 했습니다. 저의 가장 좋아하는 매체는 연필이었는데, 이는 다양한 선과 음영을 자유롭게 실험할 수 있게 해 주었습니다. 또한, 어린 시절부터 사진에 관심을 가져 카메라로 주변 세계를 탐구하고, 새로운 장면을 포착하며, 빛과 색상의 미묘한 차이를 잡아내기 위해 노력했습니다.

미술 외에도 저는 글쓰기에 푹 빠졌습니다. 어릴 때부터 일기장을 꾸준히 써왔고, 상상력을 발휘해 이야기를 지어내는 데 몰두하곤 했습니다. 또한 과학소설(sci-fi)과 판타지 같은 장르 소설을 읽는 것을 좋아했는데, 이는 저에게 우주와 다른 세계에 대한 호기심을 불러일으켰습니다.

제 관심사의 또 다른 중요한 분야는 자연 세계였습니다. 어릴 적부터 야외 활동을 즐겼으며, 주변 식물과 동물을 관찰하는 시간을 자주 가졌습니다. 나중에는 환경 운동에 관심을 가지게 되어 청소년 시절 동안 환경보호 활동을 적극적으로 지지하고 참여하게 되었습니다.


In [104]:
question1 = "도전이나 역경극복을 작성해주세요"
context1 = ask_question1(question1)
print(f"질문: {question1}")
print(f"답변: {context1}")

질문: 도전이나 역경극복을 작성해주세요
답변: 62. 당신의 인생에서 가장 큰 도전은 무엇입니까?
• 중년에 새로운 사업을 시작하는 것이 제 인생에서 가장 큰 도전이었습니다. 실패에 대한 두려움이 있었지만, 새로운 시작을 위해 모든 것을 걸고 도전을 감수했습니다.

63. 그 도전을 통해 무엇을 배웠습니까?
• 도전 덕분에 인내와 끈기의 중요성을 깨달았습니다. 성공은 단숨에 이루어지는 것이 아니라 꾸준한 노력과 인내가 필요하다는 것을 이해하게 되었습니다.

70. 당신이 남기고 싶은 유산은 무엇입니까?
• 아이들에게 사랑과 정직의 가치를 전하고 싶습니다. 제 삶의 경험을 통해 그들이 이 세상을 살아가는 바른 길을 찾을 수 있도록 가르치고 싶어요.
